In [1]:
import math
from typing import List, Optional, Tuple

import torch

from pytorch3d.renderer.cameras import FoVOrthographicCameras, FoVPerspectiveCameras, look_at_view_transform
from pytorch3d.vis.plotly_vis import plot_batch_individually, plot_scene

def make_cameras_dea(dist: torch.Tensor, elev: torch.Tensor, azim: torch.Tensor, fov: int = 10, znear: int = 18.0, zfar: int = 22.0, is_orthogonal: bool = False):
    assert dist.device == elev.device == azim.device
    _device = dist.device
    R, T = look_at_view_transform(dist=dist.float(), elev=elev.float() * 90, azim=azim.float() * 180)
    if is_orthogonal:
        return FoVOrthographicCameras(R=R, T=T, znear=znear, zfar=zfar).to(_device)
    return FoVPerspectiveCameras(R=R, T=T, fov=fov, znear=znear, zfar=zfar).to(_device)

In [9]:
_device = torch.device("cpu")
batch_size = 4
n_channels = 4
shape = 8
n_pts_per_ray = 16
dtype=torch.float32

In [10]:
cameras = make_cameras_dea(
    azim = 1.00 * torch.rand(batch_size, device=_device) * 2 - 1,
    elev = 0.00 * torch.ones(batch_size, device=_device),
    dist = 6.00 * torch.ones(batch_size, device=_device) 
) 

print(cameras.R.shape)
print(cameras.T.shape)

torch.Size([4, 3, 3])
torch.Size([4, 3])


In [11]:
cam_dict = {f"cam{idx}": cameras[[idx]] for idx in range(batch_size)}

In [12]:

fig = plot_scene(
    plots = {
        "Cameras": {
            **cam_dict,
            # **ray_dict
        },
    }, 
    # lighting=Lighting(
    #     ambient=0.1,
    #     diffuse=0.5,
    #     fresnel=0.0,
    #     specular=1.0,
    #     roughness=0.9,
    #     facenormalsepsilon=1e-6,
    #     vertexnormalsepsilon=1e-12,
    # ),
    ncols=2,
    camera_scale=1.,
    xaxis = {
        'range': [-10, 10],
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': False,  # numbers below
    }, 
    yaxis = {
        'range': [-10, 10],
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': False,  # numbers below
    }, 
    zaxis = {
        'range': [-10, 10],
        'showgrid': False, # thin lines in the background
        'zeroline': False, # thick line at x=0
        'visible': False,  # numbers below
    }, 
    pointcloud_marker_size = 1,
)
fig.update_layout(
    autosize=True,
    width=800,
    height=800,
)
fig.show()
